In [51]:
import pandas as pd
import numpy as np
import ocr_parser
from statistics import median
import re

In [121]:
entry_data = pd.read_csv("Bill.com/Users.csv")
entry_data

,documentid,paymentid,amount,date,vendor_name,vendor_address
0,00d0624439175,00p0535380313,72.90,2916-4-17,MR. D.I.Y. (KUCHZAI) SDNBHD,"LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERI..."
1,00d0654270874,00p0432205035,43.90,2016-6-7,FOUR QUARTERS SDN BHD,NO.28G JLN SS21/35 DAMANSARA UTAMA PETALING JA...
2,00d0403711886,00p0174820163,32.23,2018-10-1,SYARIKAT PERNIAGAAN GIJ KEE,"NO 290, JALAN AIR PANAS, SETAPAK, 53020, KUALA..."
3,00d0956164753,00p0943784617,503.10,2018-3-23,TIMELESS KITCHENETE SDN BHD,"LLT 50, FLOOR T2,SKY AVENUE GENTING HIGHLANDS ..."
4,00d0667839698,00p0762437501,1110.00,2016-2-28,SIN THEY & COMPANY,"NO.54, BATU 5,JALAN IPOH, KG.BATU, 5110 KUALA ..."
...,...,...,...,...,...,...
494,00d0223498663,00p0611347643,29.30,2016-12-25,GOLDEN ARCHES RESTAURANTS SDN BHD,"LEVEL 6, BANGUNAN TH, DAMANSARA UPTOWN3 NO.3, ..."
495,00d0206227401,00p0603265655,8.70,2017-8-7,SANYU STATIONERY SHOP,"NO. 31G&3G, JALAN SETIA INDAH X ,U13/X 40170 S..."
496,00d0898930568,00p0751192439,2.50,2018-1-24,99 SPEED MART S/B,"LOT P.T. 281 1, JALAN ANGSA, TAMAN BERKELEY 41..."
497,00d0397644256,00p0962484273,63.80,2018-2-22,BILLION SIX ENTERPRISE,"NO 3, JALAN TAMAN JAS'A 2; . SECTION U6, 40150..."


In [3]:
ocr_files = ocr_parser.parse_files()

In [5]:
ocr_file_keys = sorted(list(ocr_files))

In [189]:
pattern = re.compile("-{0,1}\d*\.{0,1}\d\d+$")

totals = []
for key in ocr_file_keys:
    ocr_file = ocr_files[key]
    total_lines = []
    for line_idx in range(len(ocr_file)):
        line = ocr_file.iloc[line_idx].TEXT.upper()
        words = line.split()
        # print(words)
        if (line.find("TOTAL") != -1) or ("NET" in words) or ("AMT" in words) or ("DUE" in words):
            # print(line)
            coords = ocr_file.iloc[line_idx].values[:8]
            total_y = (coords[1] + coords[7]) / 2
            total_lines.append(total_y)

    # print(total_lines)
    
    numbers = []
    for line_idx in range(len(ocr_file)):
        words = ocr_file.iloc[line_idx].TEXT.split()
        for word in words:
            matches = pattern.search(word)
            if matches:
                coords = ocr_file.iloc[line_idx].values[:8]
                num_y = (coords[1] + coords[7]) / 2
                for total_y in total_lines:
                    if np.abs(num_y - total_y) < 15:
                        candidate = float(matches.group(0))
                        numbers.append(candidate)


    # print(numbers)
    numbers = sorted(numbers)[-5:]
    # print(numbers)
    if len(numbers) != 0:
        totals.append(median(numbers))
    else:
        totals.append(None)

In [184]:
total_df = pd.DataFrame(data={
    "Document": ocr_file_keys,
    "Total": totals
})
total_df.to_csv("ocr_totals.csv")

In [185]:
correct = 0
wrong = 0
for entry_idx in range(len(entry_data)):
    entry = entry_data.iloc[entry_idx]
    entry_ocr = entry.documentid
    if entry_ocr in ocr_file_keys:
        row = total_df[total_df.Document == entry_ocr].iloc[0]
        if row.Total != None:
            if row.Total == entry.amount:
                correct += 1
                continue
            wrong += 1
            print(entry_ocr, row.Total, entry.amount)

print("correct:", correct)
print("wrong:", wrong)
print("accuracy:", correct / (correct + wrong))

00d0624439175 12.1 72.9
00d0403711886 22.0 32.23
00d0841246769 7.1 7.7
00d0320587587 20.0 28.8
00d0330533884 27.45 50.6
00d0694627982 96.0 101.75
00d0447521528 16.485 3.2
00d0546591203 140.0 2148.4
00d0796308197 28.6 28.0
00d0992127574 14.53 15.4
00d0961798581 9.995 20.0
00d0267909555 42.48 42.5
00d0280864237 165.0 174.9
00d0145695018 30.24 31.88
00d0761346037 758.7 78.7
00d0502129574 47.225 48.35
00d0686078737 nan 6.0
00d0989196649 2.0 2.12
00d0130065999 nan 0.0
00d0682329345 181.98 182.0
00d0458708415 23.64 36.3
00d0270398034 44.76 58.86
00d0234070159 19.34 20.5
00d0533588228 8.2 88.2
00d0749666123 6.6 7.0
00d0630325043 37.5 39.75
00d0484942812 nan 99.0
00d0708429922 7.92 14.74
00d0920231897 5.0 65.0
00d0855903342 213.0 2133.0
00d0212522180 144.68 153.35
00d0806991732 33.96 62.6
00d0530467663 4.51 4.0
00d0288011947 nan 1.1
00d0837897960 50.675 53.5
00d0266548523 99.8 99.78
00d0114026522 12.0 12.72
00d0760612254 5.9 11.8
00d0975647472 21.96 339.87
00d0749169797 25.56 41.44
00d05826817